For text classification there's

- binary classification (and specifically sentiment analyis)
- multi-class classification
- and for scoring or ranking docs, there's regression (Resserria)


sententence pair tasks such as semantic similairity and textual entailment

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_path = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path, 
    id2label={0: 'NEG', 1: 'POS'}, 
    label2id={"NEG": 0, "POS": 1})

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [ ]:
from datasets import load_dataset

imdb_train= load_dataset('imdb', split="train[:2000]+train[-2000:]")
imdb_test= load_dataset('imdb', split="test[:500]+test[-500:]")
imdb_val= load_dataset('imdb', split="test[500:1000]+test[-1000:-500]")
imdb_train.shape, imdb_test.shape, imdb_val.shape

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


((4000, 2), (1000, 2), (1000, 2))

In [ ]:
# now tokenize for training
tknize = lambda e: tokenizer(e['text'], padding=True, truncation=True)
enc_train = imdb_train.map(tknize, batched=True, batch_size=1000)
enc_test = imdb_test.map(tknize, batched=True, batch_size=1000)
enc_val = imdb_val.map(tknize, batched=True, batch_size=1000)

Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-ee1693c1a4681e24.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-6c7ea317d8e73ef2.arrow


In [ ]:
import pandas as pd
pd.DataFrame(enc_train)

,attention_mask,input_ids,label,text
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 22953, 2213, 4381, 2152, 2003, 1037, 947...",1,Bromwell High is a cartoon comedy. It ran at t...
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 11573, 2791, 1006, 2030, 2160, 24913, 20...",1,Homelessness (or Houselessness as George Carli...
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 8235, 2058, 1011, 3772, 2011, 23920, 575...",1,Brilliant over-acting by Lesley Ann Warren. Be...
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2023, 2003, 4089, 1996, 2087, 2104, 9250...",1,This is easily the most underrated film inn th...
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2023, 2003, 2025, 1996, 5171, 11463, 837...",1,This is not the typical Mel Brooks film. It wa...
...,...,...,...,...
3995,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2875, 1996, 2203, 1997, 1996, 3185, 1010...",0,"Towards the end of the movie, I felt it was to..."
3996,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2023, 2003, 1996, 2785, 1997, 3185, 2008...",0,This is the kind of movie that my enemies cont...
3997,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 2387, 1005, 6934, 1005, 2197, 2305...",0,I saw 'Descent' last night at the Stockholm Fi...
3998,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2070, 3152, 2008, 2017, 4060, 2039, 2005...",0,Some films that you pick up for a pound turn o...


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./MyIMDBModel',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy='steps',
    save_strategy='steps',
    fp16=cuda.is_available(),
    load_best_model_at_end=True
)

Text classifications are usually evalutated with micro-averaging or macro-averaging F1

macro-averaging gives euql weight to each class
micro-averaging is # times model is correct / total # of decisions

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=enc_train,
    eval_dataset=enc_val,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 750
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adrianmoses (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.681800,0.601343,0.836000,0.835994,0.836048,0.836000
100,0.497500,0.412913,0.805000,0.801270,0.829757,0.805000
150,0.347900,0.293118,0.886000,0.886000,0.886006,0.886000
200,0.315500,0.321727,0.889000,0.888438,0.897005,0.889000
250,0.305300,0.311679,0.875000,0.874293,0.883632,0.875000
300,0.188400,0.270497,0.910000,0.909987,0.910236,0.910000
350,0.281900,0.282820,0.895000,0.894681,0.899838,0.895000
400,0.165000,0.302467,0.909000,0.908989,0.909198,0.909000
450,0.184000,0.300530,0.909000,0.908985,0.909277,0.909000
500,0.157100,0.312986,0.916000,0.915988,0.916240,0.916000


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to ./MyIMDBModel/checkpoint-50
Configuration saved in ./MyIMDBModel/checkpoint-50/config.json
Model weights saved in ./MyIMDBModel/checkpoint-50/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to ./MyIMDBModel/checkpoint-100
Configuration saved in ./MyIMDBModel/checkpoint-100/config.json
Model weights saved in ./MyIMDBModel/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
****

In [ ]:
q = [trainer.evaluate(eval_dataset=data) for data in [enc_train, enc_val, enc_test]]
pd.DataFrame(q, index=["train", "val", "test"]).iloc[:, :5]

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 16


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.138274,0.9535,0.953499,0.953537,0.9535
val,0.270497,0.9100,0.909987,0.910236,0.9100
test,0.296340,0.8980,0.897993,0.898102,0.8980


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs

In [ ]:
def get_prediction(text):
    inputs = tokenizer(text, padding=True, truncation=True,
                      max_length=250, return_tensors="pt").to(device)
    outputs = model(inputs["input_ids"].to(device), inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

In [ ]:
text = "I didn't like the move it bored me"
get_prediction(text)[1].item()

0

In [ ]:
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

Saving model checkpoint to MyBestIMDBModel
Configuration saved in MyBestIMDBModel/config.json
Model weights saved in MyBestIMDBModel/pytorch_model.bin
tokenizer config file saved in MyBestIMDBModel/tokenizer_config.json
Special tokens file saved in MyBestIMDBModel/special_tokens_map.json


('MyBestIMDBModel/tokenizer_config.json',
 'MyBestIMDBModel/special_tokens_map.json',
 'MyBestIMDBModel/vocab.txt',
 'MyBestIMDBModel/added_tokens.json',
 'MyBestIMDBModel/tokenizer.json')

In [ ]:
from transformers import pipeline
model = DistilBertForSequenceClassification.from_pretrained("MyBestIMDBModel")
tokenizer = DistilBertTokenizerFast.from_pretrained("MyBestIMDBModel")
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


loading configuration file MyBestIMDBModel/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEG",
    "1": "POS"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEG": 0,
    "POS": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.11.3",
  "vocab_size": 30522
}

loading weights file MyBestIMDBModel/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassific

In [ ]:
nlp("the move was very impressive")

[{'label': 'POS', 'score': 0.9832347631454468}]

In [ ]:
nlp("the text of the picture was very poor")

[{'label': 'NEG', 'score': 0.9751225709915161}]